## 1. Dataset cleaning

Package set up

In [1]:
%pip install chars2vec
%pip install keras
%pip install tensorflow
%pip install sklearn
%pip install geopy
%pip install reverse_geocoder

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.mode.chained_assignment = None
from utilities import basic_cleaning, restaurant_name_to_group_id, split_by_geo_locations, get_city_state_country

## Import dataset

In [3]:
#sep to read ctrl-a as delimiter csv
#Cut the first 20k due to limited computing power
df = pd.read_csv('css_public_all_ofos_locations.csv', sep='\x01')[:20000]
df.head()

/Users/belu/.pyenv/versions/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,4,5,6,8,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
0,9744,Loco Coco,caviar,NaN,40.765004,-73.965961,NaN,US,True,loco_coco,NaN,NaN,NaN
1,4903,Musubi,caviar,NaN,45.504641,-122.643806,NaN,US,False,musubi,NaN,NaN,NaN
2,4723,The Bakers' Lounge,caviar,NaN,38.924559,-76.990115,NaN,US,False,the_bakers_lounge,NaN,NaN,NaN
3,4128,Parm - Battery Park City,caviar,NaN,40.71418,-74.015568,NaN,US,True,parm_-_battery_park_city,NaN,NaN,NaN
4,5690,Derek Test Merchant,caviar,NaN,1.0,2.0,NaN,US,False,derek_test_merchant,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      20000 non-null  object 
 1   name               20000 non-null  object 
 2   platform           20000 non-null  object 
 3   sub_platform       2 non-null      object 
 4   latitude           20000 non-null  object 
 5   longitude          20000 non-null  object 
 6   city               0 non-null      object 
 7   country            20000 non-null  object 
 8   active             20000 non-null  object 
 9   standardized_name  20000 non-null  object 
 10  restaurant_chain   0 non-null      object 
 11  delivery_radius    0 non-null      object 
 12  geom               0 non-null      float64
dtypes: float64(1), object(12)
memory usage: 2.0+ MB


## Data preprocessing

1) Drop duplicates based on 'name', 'platform', 'latitude', 'longitude'
<br>
2) 'Name' column: convert to lower case, trimmed white spaces and remove special characters
<br>
3) Format the 'active' column
<br>
4) Remove na in 'latitude', 'longitude', remove latitude >90

In [5]:
df = basic_cleaning(df)

original dataset shape (20000, 13)
0 null in name columns are found and removed
0 unknown in columns latitude/logitude are found and removed
1392 duplicated rows are found and removed
0 unknown coordinates are found and removed
clean dataset shape (18604, 13)


In [6]:
restaurant_names = df['name'].to_numpy()
restaurant_names

array(['loco coco', 'musubi', "the bakers' lounge", ...,
       "wok 'n roll catering", "rotolo's pizza", 'pines of italy'],
      dtype=object)

# 2. Clustering to group restaurants with similar names

### Converting raw text into a numerical representation

In [7]:
import chars2vec
import keras
import tensorflow.keras

# extract features from restaurant names
c2v_model = chars2vec.load_model('eng_50')
word_features = c2v_model.vectorize_words(restaurant_names)
word_features

2022-09-09 14:19:23.887511: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


array([[ 0.550715  , -0.12793979,  0.5607905 , ...,  0.09104574,
         0.9665262 , -0.19177836],
       [ 0.355227  , -0.38183346, -0.09432247, ...,  0.85210216,
        -0.47786507,  0.45215708],
       [ 0.53569144, -0.5265404 , -0.966452  , ...,  0.6271233 ,
         0.32814574, -0.76336664],
       ...,
       [-0.12941451, -0.804009  , -0.96525055, ...,  0.64572597,
         0.6509175 , -0.5678552 ],
       [ 0.6052803 , -0.15172058, -0.75829977, ...,  0.57188314,
         0.65813637, -0.4801365 ],
       [-0.12954403, -0.7627279 , -0.24654287, ..., -0.10202555,
         0.5789371 , -0.67318213]], dtype=float32)

### Clustering

In [8]:
from sklearn.cluster import AgglomerativeClustering

clustering = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.3  # smaller threshold meaning more strict similarities, and more clusters
).fit(word_features)

print(f'total clusters: {clustering.n_clusters_}')
clustering.distances_

total clusters: 16976


array([  0.        ,   0.        ,   0.        , ...,  76.81390656,
       110.24879568, 149.5172345 ])

### Append group id based on cluster results

In [9]:
restaurant_name_to_group_id(df, restaurant_names, clustering)
df.shape

(18604, 14)

### Within cluster groups, calculate distance between each other, drop duplicates if distance less than 200m 

In [10]:
df = df.groupby('group_id').apply(split_by_geo_locations).reset_index(drop=True)
df.shape

(18356, 14)

### Append city, state and country based on coordinates

In [11]:
df_ready = get_city_state_country(df)
df_ready

Loading formatted geocoded file...


,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom,group_id,state
0,16705,mr goodcents subs & pastas,delivery.com,NaN,38.897564,-94.779344,Johnson County,US,True,mr_goodcents_subs_pastas,NaN,NaN,NaN,0,Kansas
1,7166,mr. goodcents subs & pastas,delivery.com,NaN,33.407333,-111.946760,Maricopa County,US,True,mr_goodcents_subs_pastas,NaN,NaN,NaN,0,Arizona
2,9812,the french bakery - 112th ave [disabled],caviar,NaN,1.000000,-1.000000,,GH,False,the_french_bakery_-_112th_ave_disabled,NaN,NaN,NaN,1,Western
3,9904,the french bakery - 112th ave [disabled!],caviar,NaN,47.618223,-122.191230,King County,US,False,the_french_bakery_-_112th_ave_disabled,NaN,NaN,NaN,1,Washington
4,87772,k & l delicatessen,delivery.com,NaN,40.749216,-73.889470,Queens County,US,True,k_l_delicatessen,NaN,NaN,NaN,2,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18351,85440,le's restaurant- allston,delivery.com,NaN,42.352947,-71.131117,Norfolk County,US,True,le_s_restaurant-_allston,NaN,NaN,NaN,16971,Massachusetts
18352,62212,organicdirect,delivery.com,NaN,40.638689,-73.926104,Kings County,US,False,organicdirect,NaN,NaN,NaN,16972,New York
18353,68647,pines of florence,delivery.com,NaN,38.917776,-77.047258,,US,True,pines_of_florence,NaN,NaN,NaN,16973,"Washington, D.C."
18354,64778,locos grill & pub,delivery.com,NaN,33.826853,-84.331843,DeKalb County,US,True,locos_grill_pub,NaN,NaN,NaN,16974,Georgia


### Save it to csv file

In [12]:
df_ready.to_csv('clean_restaurant_list_sample.csv')

## Dashboard building (to be continued...)

filter by platform, country, state, city, active
<br>
allow search by name, id

In [15]:
df_ready.columns

Index(['restaurant_id', 'name', 'platform', 'sub_platform', 'latitude',
       'longitude', 'city', 'country', 'active', 'standardized_name',
       'restaurant_chain', 'delivery_radius', 'geom', 'group_id', 'state'],
      dtype='object')

In [17]:
df_ready.active.unique()

array([ True, False])

In [26]:
import plotly.express as px
token = ''
px.set_mapbox_access_token(token)
data = df_ready[df_ready['active'].notna()]

fig = px.scatter_mapbox(data, lat="latitude", lon="longitude", color="active", hover_name="name", 
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=3, title='Restaurant Map')

# Add dropdowns
button_layer_height = 1.2
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=['platform', 'all'],
                    label='all',
                    method='restyle'
                ),
                dict(
                    args=['platform', 'delivery.com'],
                    label='delivery.com',
                    method='restyle'
                ),
                dict(
                    args=['platform', 'caviar'],
                    label='caviar',
                    method='restyle'
                ),
                dict(
                    args=['platform', 'grubhub'],
                    label='grubhub',
                    method='restyle'
                )
            ]),
            direction='down',
            pad={'r':10, 't':10},
            showactive=True,
            x=0,
            xanchor='left',
            y=button_layer_height,
            yanchor='top'
        ),
        dict(
            buttons=list([
                dict(
                    args=['active', 'True'],
                    label='active',
                    method='restyle'
                ),
                dict(
                    args=['active', 'False'],
                    label='inactive',
                    method='restyle'
                )
            ]),
            direction='down',
            pad={'r':10, 't':10},
            showactive=True,
            x=0.58,
            xanchor='left',
            y=button_layer_height,
            yanchor='top'
        )
    ]
)



fig.update_layout(mapbox_style='open-street-map')
fig.show()